In [140]:
%reset -f

from pandasgui import show

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functions import *
from sklearn.model_selection import train_test_split
from statsmodels.stats.diagnostic import acorr_ljungbox

%load_ext autoreload
%autoreload 2

# Load data
data = pd.read_csv('Data/non_nan_data.csv')
data = data.drop(['sun_mean', 'wind_mean', 'temp_mean'], axis=1)
data = data.drop(['DK1_spot'], axis=1)

# Split data when date < 2020-01-01 21:00:00
data_train = data[data['DATE'] < '2020-01-01'].drop(['DATE'], axis=1).to_numpy()
data_test = data[data['DATE'] >= '2020-01-01'].drop(['DATE'], axis=1).to_numpy()

data_test

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


array([[237.330002,   0.      ,   0.      , ...,   5.1     ,   6.6     ,
          2.1     ],
       [235.830002,   0.      ,   0.      , ...,   5.1     ,   7.6     ,
          3.      ],
       [233.660004,   0.      ,   0.      , ...,   7.7     ,   8.7     ,
          3.3     ],
       ...,
       [297.820007,   0.      ,   0.      , ...,   7.7     ,   6.      ,
          7.1     ],
       [261.040009,   0.      ,   0.      , ...,   8.8     ,   4.9     ,
          7.2     ],
       [195.839996,   0.      ,   0.      , ...,   9.3     ,   5.2     ,
          6.6     ]])

In [143]:
show(data[data['DATE'] >= '2020-01-01'].drop(['DATE'], axis=1))

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
C:\Users\Peter\AppData\Local\Temp\ipykernel_10576\3891989758.py:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\Peter\AppData\Local\Temp\ipykernel_10576\3891989758.py:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\Peter\AppData\Local\Temp\ipykernel_10576\3891989758.py:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\Peter\AppData\Local\Temp\ipykernel_10

## Baseline

In [149]:
p = 1

y_train = data_train[:, 0].reshape(-1, 1)[1::]
y_test = data_test[:, 0].reshape(-1, 1)[1::]

weather_train = data_train[:, 1:-2][:-1, :]
weather_test = data_test[:, 1:-2][:-1, :]

hour_dummies_train = sdummy(len(data_train), 24)[:-1, :]
hour_dummies_test = sdummy(len(data_test), 24)[:-1, :]

all_train = np.concatenate((weather_train, hour_dummies_train), axis=1)
all_test = np.concatenate((weather_test, hour_dummies_test), axis=1)


con = 0
tr = 0
exog = 0
# exog = weather
# exog = hour_dummies_train
# exog = all_train

Beta, SEbeta, _, Pvalue, tratioBeta, res_baseline, _, _, aiccrit, hqccrit, siccrit = VARlsExog(y_train, p, con, tr, exog)

# # Collect beta, SEbeta and tratioBeta in a single dataframe
results = pd.DataFrame()
results['Beta'] = Beta.reshape(-1, order='F')


X_test = np.column_stack([lagmatrix(y_test, p)])

y_pred = X_test @ Beta

smape_baseline = smape(y_test[p::], y_pred[p::])

print(f'SMAPE baseline: {smape_baseline}')

SMAPE baseline: 18.333736337737868


## Baseline with 24 lags

In [150]:
p = 24

y_train = data_train[:, 0].reshape(-1, 1)[1::]
y_test = data_test[:, 0].reshape(-1, 1)[1::]

weather_train = data_train[:, 1:-2][:-1, :]
weather_test = data_test[:, 1:-2][:-1, :]

hour_dummies_train = sdummy(len(data_train), 24)[:-1, :]
hour_dummies_test = sdummy(len(data_test), 24)[:-1, :]

all_train = np.concatenate((weather_train, hour_dummies_train), axis=1)
all_test = np.concatenate((weather_test, hour_dummies_test), axis=1)


con = 0
tr = 0
exog = 0
# exog = weather
# exog = hour_dummies_train
# exog = all_train

Beta, SEbeta, _, Pvalue, tratioBeta, res_baseline, _, _, aiccrit, hqccrit, siccrit = VARlsExog(y_train, p, con, tr, exog)

# # Collect beta, SEbeta and tratioBeta in a single dataframe
results = pd.DataFrame()
results['Beta'] = Beta.reshape(-1, order='F')


X_test = np.column_stack([lagmatrix(y_test, p)])

y_pred = X_test @ Beta

smape_baseline = smape(y_test[p::], y_pred[p::])

print(f'SMAPE baseline: {smape_baseline}')

SMAPE baseline: 19.13263847173096


## Extended baseline with weather data

In [148]:
p = 1

y_train = data_train[:, 0].reshape(-1, 1)[1::]
y_test = data_test[:, 0].reshape(-1, 1)[1::]

weather_train = data_train[:, 1:-2][:-1, :]
weather_test = data_test[:, 1:-2][:-1, :]

hour_dummies_train = sdummy(len(data_train), 24)[:-1, :]
hour_dummies_test = sdummy(len(data_test), 24)[:-1, :]

all_train = np.concatenate((weather_train, hour_dummies_train), axis=1)
all_test = np.concatenate((weather_test, hour_dummies_test), axis=1)


con = 0
tr = 0
# exog = weather_train
# exog = hour_dummies_train
exog = all_train

Beta, SEbeta, _, Pvalue, tratioBeta, _, _, _, aiccrit, hqccrit, siccrit = VARlsExog(y_train, p, con, tr, exog)

# # Collect beta, SEbeta and tratioBeta in a single dataframe
results = pd.DataFrame()
results['Beta'] = Beta.reshape(-1, order='F')


X_test = np.column_stack([lagmatrix(y_test, p), all_test])

y_pred = X_test[p::, :] @ Beta

smape_baseline_extended = smape(y_test[p::, :] , y_pred)

print(f'SMAPE baseline extended: {smape_baseline_extended}')

SMAPE baseline extended: 18.523369269030294
